In [10]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 6.2 MB/s  0:00:01m 6.3 MB/s eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [14]:
#input to search
search_box_text = 'sports shoes for women'
website_link = 'https://www.flipkart.com/'

#initiating the browser
#session start time
session_start_time = datetime.now().time()
print(f"session start time: {session_start_time}")

#starting the bowser
driver = webdriver.Chrome()
driver.get(website_link)
driver.maximize_window()

print('waiting for search input...')
search_input = WebDriverWait(driver,120).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[autocomplete = "off"]')))

print('typing in search input...')
search_input.send_keys(search_box_text)

print('submitting search form...')
search_input.send_keys(Keys.RETURN)

print('Waiting for search results...')
WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[target="_blank"]')))

print('collecting pagination links')

#we want first 25 pages
#logic: let's get the first page pagination link and append the number in the end of 25 pages and store in a list 
all_pagination_links = []

first_page = driver.find_elements(By.CSS_SELECTOR,'nav a')[0]
first_page_link = first_page.get_attribute('href')
all_pagination_links.append(first_page_link)

for i in range(2,26):
    new_pagination_link = first_page_link[: -1] + str(i)
    all_pagination_links.append(new_pagination_link)

print('pagination links count:', len(all_pagination_links))
print('all pagination links:', all_pagination_links)

print("collecting product detail page links")
all_product_links=[]

for link in all_pagination_links:
    try:
        driver.get(link)
        WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CLASS_NAME, 'rPDeLR')))
    except Exception as e:
        print(f"Skipping link {link}: {e}")
        continue # Moves to the next link without crashing

    all_products = driver.find_elements(By.CLASS_NAME,'rPDeLR')
    all_links = [element.get_attribute('href') for element in all_products]

    all_pagination_links.extend(all_links)

    df_product_links= pd.DataFrame(all_product_links,columns=['product_links'])
    df_product_links= df_product_links.drop_duplicates(subset=['product_links'])

    df_product_links.to_csv('flipkart_product_links.csv', index=False)

    driver.close()
    session_end_time = datetime.now().time()
    print(f"session end time:{session_end_time}")

session start time: 23:53:32.056931
waiting for search input...
typing in search input...
submitting search form...
Waiting for search results...
collecting pagination links
pagination links count: 25
all pagination links: ['https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=1', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=2', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=3', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=4', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=5', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&ot